A Tool Calling Agent refers to an AI system
or framework that dynamically invokes external tools or APIs to accomplish
tasks as part of its decision making process.Instead of relying soley on internal reasoning or knowledge,the agent calls specific tools to perform actions like calculations,searches, or other operations.This type of agent is often used in realworld applications where tasks 
require interaction with external systems.

In [34]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [44]:
tanvily_api_key=os.getenv("TANVILY_API_KEY")
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
os.environ['google_api_key']=GOOGLE_API_KEY
os.environ['tavily_api_key']=tanvily_api_key

In [45]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [46]:
search=TavilySearchResults()

In [47]:
search.invoke("What is weather in lucknow?")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'Lucknow', 'region': 'Uttar Pradesh', 'country': 'India', 'lat': 26.85, 'lon': 80.917, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1731650211, 'localtime': '2024-11-15 11:26'}, 'current': {'last_updated_epoch': 1731649500, 'last_updated': '2024-11-15 11:15', 'temp_c': 28.7, 'temp_f': 83.6, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 7.6, 'wind_kph': 12.2, 'wind_degree': 289, 'wind_dir': 'WNW', 'pressure_mb': 1014.0, 'pressure_in': 29.96, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 25, 'cloud': 0, 'feelslike_c': 27.1, 'feelslike_f': 80.7, 'windchill_c': 28.7, 'windchill_f': 83.6, 'heatindex_c': 27.1, 'heatindex_f': 80.7, 'dewpoint_c': 6.7, 'dewpoint_f': 44.1, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 4.6, 'gust_mph': 8.7, 'gust_kph': 14.1}}"},
 {'url': 'https://www.hindustantimes.com/cities/lucknow-news/lucknow-weather-today-a

In [48]:
tools=[search]

In [52]:
from langchain import hub

In [55]:
prompt=hub.pull("hwchase17/openai-functions-agent")

a:\LANGCHAIN\langchainenv\lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [57]:
print(prompt)

input_variables=['agent_scratchpad', 'input'] optional_variables=['chat_history'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} partial_variables={'chat_history': []} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], te

In [58]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [59]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

`Agent sctrachpad`: The agent does not call a tool just once to get the desird answer , but has structure that calls tools repeatedly until desired anser is obtaines. each time you calll a tool , what the previous call was like, information about the previous call, and the result are stored in this field.

In [60]:
from langchain.agents import create_tool_calling_agent
agent=create_tool_calling_agent(llm,tools,prompt)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [62]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [64]:
agent_executor.invoke({"input":"hello how are you?"})



> Entering new AgentExecutor chain...
I'm doing well, thank you for asking! How can I help you today?


> Finished chain.


{'input': 'hello how are you?',
 'output': "I'm doing well, thank you for asking! How can I help you today?\n"}

In [68]:
agent_executor.invoke({"input":"who won t20 world 2024?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Who won the T20 World Cup 2024?'}`


[{'url': "https://en.wikipedia.org/wiki/2024_ICC_Men's_T20_World_Cup_final", 'content': "The 2024 ICC Men's T20 World Cup final was a Twenty20 International cricket match played at Kensington Oval in Bridgetown, Barbados on 29 June 2024 to determine the winner of the 2024 ICC Men's T20 World Cup. [1] [2] It was played between South Africa and India.[3]India defeated South Africa by 7 runs to win their second T20 World Cup title. [4] Virat Kohli was named Player of the Match for"}, {'url': 'https://indianexpress.com/article/sports/cricket/india-wins-t20-world-cup-2024-live-updates-indian-cricket-team-celebration-virat-kohli-rohit-sharma-retirement-9423593/', 'content': "Virat Kohli and Rohit Sharma with T20 World Cup 2024. (Agencies) India Wins T20 World Cup Final highlights: Following Team India's historic triumph, the BCCI has announced an eye-popping R

{'input': 'who won t20 world 2024?',
 'output': 'India won the T20 World Cup in 2024, defeating South Africa by 7 runs in the final held in Barbados.\n'}